# Binary / Binomial Data (replicating [the R package's vignette](https://github.com/stan-dev/rstanarm/blob/master/vignettes/binomial.Rmd))

In [ ]:
%load_ext autoreload
%autoreload 2

from bayesglm import stan_glm, priors, family
import pandas as pd
from ggplot import gg, ggplot_notebook
import numpy as np
from scipy.special import logit, expit
from rpy2 import robjects

In [ ]:
# Load data and create dist100 variable 
wells = pd.read_csv("data/wells.csv")
wells['dist100'] = wells.dist / 100

In [ ]:
wells.head()

In [ ]:
p = gg.ggplot(wells) + \
    gg.aes_string(x = "dist100") + \
    gg.geom_histogram() + \
    gg.facet_grid("switch ~ .", scales = "free_y")
    
# p = gg.ggplot(wells) + \
#     gg.aes_string(x = "dist100", fill = "factor(switch)") + \
#     gg.geom_histogram(gg.aes_string(y = "..density..", group = "factor(switch)"), alpha = .3) + 
#     gg.stat_bin()
    

In [ ]:
ggplot_notebook(p)

In [ ]:
t_prior = priors.StudentTPrior(7, 0, 2.5)


In [ ]:
fit1 = stan_glm("switch ~ dist100", wells, 
                 family = family.bernoulli_logit(), 
                 priors = {"Intercept": t_prior, "dist100": t_prior})

In [ ]:
fit1

In [ ]:
posterior_samples = fit1.extract(permuted=True)['beta']
beta_intercept, beta_dist100 = np.mean(posterior_samples, axis=0)
pr_switch = lambda dist100: expit(beta_intercept + beta_dist100 * dist100)

In [ ]:

dist100s = np.linspace(start = min(wells.dist100), stop = max(wells.dist100), num = 100)
pr_switch_df = pd.DataFrame({"dist100": dist100s, "switch": pr_switch(dist100s)})
pr_switch_df.head()

In [ ]:
p = gg.ggplot(wells) + \
    gg.aes_string(x = "dist100", y = "switch", color = "switch") + \
    gg.geom_point(position = gg.position_jitter(height = .05, width = .1)) +\
    gg.geom_line(gg.aes_string(x="dist100", y="switch"), data = pr_switch_df)
ggplot_notebook(p)


In [ ]:
fit2 = stan_glm("switch ~ dist100 + arsenic", wells, 
                 family = family.bernoulli_logit(), 
                 priors = {"Intercept": t_prior, "dist100": t_prior})

In [ ]:
fit2

In [ ]:
posterior_samples = fit2.extract(permuted=True)['beta']
beta_intercept, beta_dist100, beta_arsenic = np.mean(posterior_samples, axis=0)
pr_switch2 = lambda dist100, arsenic: expit(beta_intercept + beta_dist100 * dist100 + beta_arsenic * arsenic)

In [ ]:
# pr_switch2 <- function(x, y, ests) plogis(ests[1] + ests[2] * x + ests[3] * y)
# grid <- expand.grid(dist100 = seq(0, 4, length.out = 100), 
#                     arsenic = seq(0, 10, length.out = 100))
# grid$prob <- with(grid, pr_switch2(dist100, arsenic, coef(fit2)))
# ggplot(grid, aes(x = dist100, y = arsenic)) + 
#   geom_tile(aes(fill = prob)) + 
#   geom_point(data = wells, aes(color = factor(switch)), size = 2, alpha = 0.85) + 
#   scale_fill_gradient() +
#   scale_color_manual("switch", values = c("white", "black"), labels = c("No", "Yes"))

In [ ]:
pd.DataFrame(np.array(np.meshgrid(np.array([1,2,3]), np.array([11,12,13]), indexing="ij")).reshape(2, 9).transpose())